## Prática: Criação da configuração experimental e avaliação dos resultados

Nesta prática você irá terminar de desenvolver o código responsável por dividir o dataset em folds, métricas dos resultados e execução do método (Parte 1, no arquivo `resultados.py` e `metodo.py`), avaliação e configuração de parametros (Parte 2, no arquivo `avaliacao.py`) e, finalmente, irá aplicar isso em um cenário de classificação de segmentos de imagens de texturas diversas.

[Veja os slides sobre as métricas de avaliação](https://docs.google.com/presentation/d/1u5x2b9BxmGXAWtfe9WanBIdqrt2k2ArKuEGY5Ks-okA/edit?usp=sharing)

## Antes de começar...

Lembre-se da [aula de classes em Python](https://daniel-hasan.github.io/cefet-web-grad/classes/python2/), mais especificamente:[Propriedades](https://www.youtube.com/watch?v=ocezOnXIzrc&list=PLwIaU1DGYV6skjkahOKtpgs9bPXlrVrIp&index=7) , [objetos que podem ser chamados](https://www.youtube.com/watch?v=EXmr7zttGWE&list=PLwIaU1DGYV6skjkahOKtpgs9bPXlrVrIp&index=9), [classes e herança](https://www.youtube.com/watch?v=zEP8baA_1lQ&list=PLwIaU1DGYV6skjkahOKtpgs9bPXlrVrIp&index=10)

**Dependencias: ** Para esta prática, você deverá instalar o optuna e o hiplot:

In [1]:
!pip install optuna hiplot numpy

  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/69/60/87a06ef66b34cbe2f2eb0ab66f003664404a7f40c21403a69fad7e28a82b/optuna-3.3.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/863.2 kB ? eta -:--:--
     ---- --------------------------------- 112.6/863.2 kB 3.3 MB/s eta 0:00:01
     ------------ ------------------------- 276.5/863.2 kB 3.4 MB/s eta 0:00:01
     ------------------ ------------------- 409.6/863.2 kB 3.7 MB/s eta 0:00:01
     ---------------------- --------------- 512.0/863.2 kB 3.2 MB/s eta 0:00:01
     ----------------------------- -------- 675.8/863.2 kB 3.1 MB/s eta 0:00:01
     ------------------------------------ - 819.2/863.2 kB 3.1 MB/s eta 0:00:01
     -------------------------------------- 863.2/863.2 kB 2.9 MB/s eta 0:00:00
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/a2/8b/46919127496036c8e990b2b236454a0d8655fd46e1df2fd35610a9cbc842/alem

DEPRECATION: Loading egg at c:\development\python3.11.3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


Lembre-se do método *drop* e *sample* da prática passada ;)

## Parte 1 - Métricas de resultado, método de aprendizado e divisão por folds

**Atividade 1**: Primeiramente, analise a classe `Resultado` que possui os seguintes atributos/propriedades: 
    
   - **mat_confusão**: Retorna a matriz de confusão correpondente. Analise o código e entenda o que representa a linha e a coluna dessa matriz.
    
   - **precisao**:A partir da matriz de confusão, calcula a precisão por classe. Cada indice é o rótulo da classe. Caso o número de elementos previstos com uma determinada classe qualquer `c` seja zero, então `precisao[c] = 0`. Nesses casos, é [lançado um warning](https://docs.python.org/3.7/library/warnings.html) da classe `UndefinedMetricWarning` com uma mensagem que não havia instancias previstas para essa classe.
   - **revocacao**: De forma similar à `precisao`, calcula a revocação por meio da matriz de confusão. Caso o número de elementos dessa classe seja igual a zero, então a revocação para esta class é zero e também deverá ser retornado um warning `UndefinedMetricWarning` com essa informação. 
   - **f1_por_classe**: Retorna, para cada classe, o seu valor F1. Caso a soma da precisão e revocação dessa classe seja zero, deverá ser retornado zero.

Você deverá implementar as seguintes propriedades: 

   - **macro_f1**: Calcula a média do f1 por classe. O método [`np.average`](https://numpy.org/doc/stable/reference/generated/numpy.average.html) pode ajudar.
   - **acuracia**: Calcula a acurácia  por meio da matriz de confusão.



Logo após, execute os seguintes testes automatizados: 

para validar o macro F1:

In [13]:
!python -m tests TestResultado.test_macro_f1

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


Para validar a acurácia: 

In [14]:
!python -m tests TestResultado.test_acuracia

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


**Atividade 2 - Classe ScikitLearnAprendizadoDeMaquina:** O arquivo `metodo.py` é o arquivo que possui os métodos de aprendizado de máquina. A classe `MetodoAprendizadoMaquina` é a classe abstrata para armazenar um método de aprendizado de máquina. Cada instancia, representa um método com seus determinados parametros. A classe `ScikitLearnaprendizadoDemaquina` é responsável por implementar métodos de aprendizado de máquina da API do [Scikit Learn](http://scikit-learn.org). Cada instancia desta classe armazena o respectivo método no atributo `ml_method`.  Por exemplo: 



In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from metodo import ScikitLearnAprendizadoDeMaquina

#o primeiro método é uma arvore de decisao
clf_dtree_1 = DecisionTreeClassifier(random_state=1,min_samples_split=0.3)
metodo_1 = ScikitLearnAprendizadoDeMaquina(clf_dtree_1)

#veja que o segundo método também é uma arvore de decisão, porém, com paramtros diferentes
clf_dtree_2 = DecisionTreeClassifier(random_state=1,min_samples_split=0.2)
metodo_2 = ScikitLearnAprendizadoDeMaquina(clf_dtree_2)

#terceiro método é uma RandomForest
clf_rforest = RandomForestClassifier(min_samples_split=0.3,n_estimators=100,
                                            max_features=0.7)
metodo_3 = ScikitLearnAprendizadoDeMaquina(clf_rforest)

Você deverá implementar o método `eval` da classe `ScikitLearnAprendizadoDeMaquina`. O dado de treino está no DataFrame `df_treino` e o dado a ser avaliado (teste ou validação) é o `df_data_to_predict`. Tais DataFrames são compostos por um conjunto de colunas que são os atributos e uma coluna que é a classe alvo (o nome da coluna da classe está armazenado em `col_classe`).  Veja o exemplo abaixo:

In [16]:
import pandas as pd
df_treino = pd.DataFrame([["sim","sim","não","não"],
                           ["não","não","sim","sim"],
                         ["sim","não","não","sim"]],
                        columns=["chuvoso","ventos fortes","ensolarado","jogar volei?"])
df_treino

,chuvoso,ventos fortes,ensolarado,jogar volei?
0,sim,sim,não,não
1,não,não,sim,sim
2,sim,não,não,sim


Neste exemplo o objetivo é verificar se é possível jogar uma partida de vôlei dependendo das situações climáticas. Neste contexto, `chuvoso`, `ventos fortes` e `ensolarado` são os atributos e `jogar volei?` é a classe. 

Assim, você deve implementar o método `eval` da classe `ScikitLearnAprendizadoDeMaquina` para treinar e avaliar. Para isso, você deverá separar a coluna que se refere a classe e as colunas que se referem aos atributos. Logo após você deverá criar o modelo e executar o método predict() para obter a predições. No final, este método retorna uma instancia da classe `Resultado`.

Execute o código abaixo para verificar o funcionamento deste método: 

In [17]:
!python -m tests MetodoTest.test_eval

Macro f1: 0.5982142857142857 Acuracia: 0.6


.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


**Atividade 3 - Criação dos folds:** O arquivo `resultado.py` possui a classe `Fold` que é responsável por armazenar o treino, teste e validação (quando existente). Essa classe possui os seguintes atributos: 

- `df_treino`: Dataframe com as instancias de treino. Cada instancia é uma linha e, suas colunas, são seus atributos e a sua classe
- `df_data_to_predict`: Dataframe com as instancias de teste, representada da mesma forma que `df_treino`
- `col_classe`: Coluna que representa a classe alvo nos DataFrames `df_treino` e `df_data_to_predict`
- `arr_folds_validacao`: vetor com os folds de validação. Os folds de validação são também instancias da classe Fold. Tais instancias são construidas a partir do treino - você irá fazer isso nesta atividade. 

**Atividade 3(a):** Primeiramente, implemente o [método estático](https://daniel-hasan.github.io/cefet-web-grad/classes/python2/#heranca) `gerar_k_folds`. A principio, ignore os parametros  `num_folds_validacao` e `num_repeticoes_validacao`. Este método divide em vários fold os dados `df_dados`. Cada fold deverá ser representado por uma instancia da classe Fold. Deve-se dividir o dataset em $k$ folds (parâmetro  `val_k`) e podem ser feitas $n$ repetições (parâmetro `num_repeticoes`). A escolha das instancias é feita sempre aleatória e, em cada repetição, todos os valores devem estar presentes  em apenas um teste. O treino ficaria com o restante dos valores. Veja abaixo um exemplo se dividirmos em três folds com duas repetições. Foi feito uma função para dividir em três folds e um exemplo em que foi gerado 2 repetições dele.  Para isso, usou-se a função [sample](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) para embaralhar os dados e um seed fixo para que sempre embaralhar da mesma forma. Esse seed é essencial para que se garanta a repodutibilidade dos experimentos. 

In [18]:
from resultado import Fold
import pandas as pd
def gera_tres_folds(df_dados, col_classe): 
    
    fold_1 = Fold(df_treino=df_dados[3:]                            ,df_data_to_predict=df_dados[0:3],col_classe=col_classe)
    fold_2 = Fold(df_treino=pd.concat([df_dados[0:3],df_dados[6:]]), df_data_to_predict=df_dados[3:6],col_classe=col_classe)
    fold_3 = Fold(df_treino=df_dados[0:6]                       , df_data_to_predict=df_dados[6:9],col_classe=col_classe)
    
    return fold_1, fold_2, fold_3

colunas = ["atributo_1","classe_alvo"]
col_classe = "classe_alvo"
df_dados = pd.DataFrame([[0,"S"],[1,"S"],[2,"N"],
                   [3,"N"],[4,"N"],[5,"S"],
                   [6,"S"],[7,"N"],[8,"S"]], columns=colunas)

In [19]:
#para ficar mais facil visualizar, comentamos o código abaixo e, assim, 
#. nao embaralhamos a primeira execução
#. com isso, na primeira execucao os valores estarão ordenados de forma crescente 
#. pois foi esta a forma que foram inseridos
#df_dados_rand = df_dados.sample(frac=1,random_state=2)
fold_1,fold_2,fold_3 = gera_tres_folds(df_dados, col_classe)
arr_folds = [fold_1,fold_2,fold_3]

#veja o dado de treino ou a ser previsto de cada fold na primeira repetição 
#..(substitua o numero de fold o ou o atributo): 
fold_3.df_data_to_predict

,atributo_1,classe_alvo
6,6,S
7,7,N
8,8,S


In [20]:
#Na segunda execução, os folds devem ser diferentes ao embaralhar
#por isso, usamos uma seed diferente
#verifique que, mesmo executando várias vezes, como o seed é fixo,
#o dataframe é sempre embaralhado da mesma forma. Isso ajuda a garantir a reprodutibilidade.
df_dados_rand = df_dados.sample(frac=1,random_state=2)
fold_1,fold_2,fold_3 = gera_tres_folds(df_dados_rand, col_classe)

#adiciona mais os tres folds  na lista
#veja em: https://www.geeksforgeeks.org/append-extend-python/
arr_folds.extend([fold_1,fold_2,fold_3])

#veja p resutado de todas as execucoes:
num_repeticoes = 2
val_k = 3
for num_repeticao in range(num_repeticoes):
    for num_fold in range(val_k):
        i = val_k*num_repeticao+num_fold
        df_treino  = arr_folds[i].df_treino
        df_to_predict  = arr_folds[i].df_data_to_predict
        qtd_treino = len(df_treino.index)
        qtd_to_predict = len(df_to_predict.index)
        print(f"Repeticao #{num_repeticao}  Fold #{num_fold} instancias no treino: {qtd_treino} teste: {qtd_to_predict}")
        print(f"\tÍndices das instancias do treino: {df_treino.index.values}")
        print(f"\tÍndices das instancias a avaliar (teste ou validação): {df_to_predict.index.values}")
        print(" ")

Repeticao #0  Fold #0 instancias no treino: 6 teste: 3
	Índices das instancias do treino: [3 4 5 6 7 8]
	Índices das instancias a avaliar (teste ou validação): [0 1 2]
 
Repeticao #0  Fold #1 instancias no treino: 6 teste: 3
	Índices das instancias do treino: [0 1 2 6 7 8]
	Índices das instancias a avaliar (teste ou validação): [3 4 5]
 
Repeticao #0  Fold #2 instancias no treino: 6 teste: 3
	Índices das instancias do treino: [0 1 2 3 4 5]
	Índices das instancias a avaliar (teste ou validação): [6 7 8]
 
Repeticao #1  Fold #0 instancias no treino: 6 teste: 3
	Índices das instancias do treino: [2 3 0 5 7 8]
	Índices das instancias a avaliar (teste ou validação): [4 1 6]
 
Repeticao #1  Fold #1 instancias no treino: 6 teste: 3
	Índices das instancias do treino: [4 1 6 5 7 8]
	Índices das instancias a avaliar (teste ou validação): [2 3 0]
 
Repeticao #1  Fold #2 instancias no treino: 6 teste: 3
	Índices das instancias do treino: [4 1 6 2 3 0]
	Índices das instancias a avaliar (teste ou va

Diferentemente da função implementada acima, o método a ser implementado por vocês deverá ter uma quantidade qualquer de repetições (parâmetro `num_repeticoes`) e folds por repetição (parâmetro `val_k`). Ao criar o fold, também é necessário saber a coluna da classe, para isso, o parametro `col_classe` armazena seu valor. Implemente o método `gerar_k_folds`. Logo após, execute o seguinte teste automatizado: 

In [21]:
!python -m tests TestFold.test_gerar_k_folds

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


**Atividade 3(b):** Agora, você deverá inicialiar o atributo `arr_folds_validacao` com um vetor de folds de validação, por meio dos dados de treino, de acordo com o número de repetições e folds passados como parametro no construtor. Para isso, invoque o método `gerar_k_folds` no construtor - note que estes folds a serem criados não possuirão validação - possuirão apenas treino e dados a serem previstos (teste). 

Logo após, faça uma pequena modificação no `gerar_k_folds`: os parametros  `num_folds_validacao` e `num_repeticoes_validacao` indicam se o fold a ser criado possuirá validação. Ao instanciar o fold, esses parametros devem ser passados para o contrutor do Fold. Teste a execução a seguir: 

In [22]:
!python -m tests TestFold.test_arr_validacao

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
